# SimpleBGC controller

Implemented using the serial API specified in www.basecamelectronics.com/serialapi/

In [12]:
import numpy as np

In [13]:
import serial
ser = serial.Serial('/dev/serial0',
                    baudrate=115200,
                    parity=serial.PARITY_NONE,
                    stopbits=serial.STOPBITS_ONE,
                    bytesize=serial.EIGHTBITS,
                    timeout=0.2)
#serial0 == '/dev/ttyS0', connected on physical pins 8 & 10.
#serial1 == '/dev/ttyAMA0', connected internally to the Bluetooth hardware.

In [14]:
L = {
    'CMD_READ_PARAMS': 82,
    'CMD_WRITE_PARAMS': 87,
    'CMD_REALTIME_DATA': 68,
    'CMD_BOARD_INFO': 86,
    'CMD_CALIB_ACC': 65,
    'CMD_CALIB_GYRO': 103,
    'CMD_CALIB_EXT_GAIN': 71,
    'CMD_USE_DEFAULTS': 70,
    'CMD_CALIB_POLES': 80,
    'CMD_RESET': 114,
    'CMD_HELPER_DATA': 72,
    'CMD_CALIB_OFFSET': 79,
    'CMD_CALIB_BAT': 66,
    'CMD_MOTORS_ON': 77,
    'CMD_MOTORS_OFF': 109,
    'CMD_CONTROL': 67,
    'CMD_TRIGGER_PIN': 84,
    'CMD_EXECUTE_MENU': 69,
    'CMD_GET_ANGLES': 73,
    'CMD_CONFIRM': 67,
    'CMD_BOARD_INFO_3': 20,
    'CMD_READ_PARAMS_3': 21,
    'CMD_WRITE_PARAMS_3': 22,
    'CMD_REALTIME_DATA_3': 23,
    'CMD_REALTIME_DATA_4': 25,
    'CMD_SELECT_IMU_3': 24,
    'CMD_READ_PROFILE_NAMES': 28,
    'CMD_WRITE_PROFILE_NAMES': 29,
    'CMD_QUEUE_PARAMS_INFO_3': 30,
    'CMD_SET_ADJ_VARS_VAL': 31,
    'CMD_SAVE_PARAMS_3': 32,
    'CMD_READ_PARAMS_EXT': 33,
    'CMD_WRITE_PARAMS_EXT': 34,
    'CMD_AUTO_PID': 35,
    'CMD_SERVO_OUT': 36,
    'CMD_I2C_WRITE_REG_BUF': 39,
    'CMD_I2C_READ_REG_BUF': 40,
    'CMD_WRITE_EXTERNAL_DATA': 41,
    'CMD_READ_EXTERNAL_DATA': 42,
    'CMD_READ_ADJ_VARS_CFG': 43,
    'CMD_WRITE_ADJ_VARS_CFG': 44,
    'CMD_API_VIRT_CH_CONTROL': 45,
    'CMD_ADJ_VARS_STATE': 46,
    'CMD_EEPROM_WRITE': 47,
    'CMD_EEPROM_READ': 48,
    'CMD_CALIB_INFO': 49,
    'CMD_BOOT_MODE_3': 51,
    'CMD_SYSTEM_STATE': 52,
    'CMD_READ_FILE': 53,
    'CMD_WRITE_FILE': 54,
    'CMD_FS_CLEAR_ALL': 55,
    'CMD_AHRS_HELPER': 56,
    'CMD_RUN_SCRIPT': 57,
    'CMD_SCRIPT_DEBUG': 58,
    'CMD_CALIB_MAG': 59,
    'CMD_GET_ANGLES_EXT': 61,
    'CMD_READ_PARAMS_EXT2': 62,
    'CMD_WRITE_PARAMS_EXT2': 63,
    'CMD_GET_ADJ_VARS_VAL': 64,
    'CMD_CALIB_MOTOR_MAG_LINK': 74,
    'CMD_GYRO_CORRECTION': 75,
    'CMD_DATA_STREAM_INTERVAL': 85,
    'CMD_REALTIME_DATA_CUSTOM': 88,
    'CMD_BEEP_SOUND': 89,
    'CMD_ENCODERS_CALIB_OFFSET_4': 26,
    'CMD_ENCODERS_CALIB_FLD_OFFSET_4': 27,
    'CMD_CONTROL_CONFIG': 90,
    'CMD_CALIB_ORIENT_CORR': 91,
    'CMD_COGGING_CALIB_INFO': 92,
    'CMD_CALIB_COGGING': 93,
    'CMD_CALIB_ACC_EXT_REF': 94,
    'CMD_PROFILE_SET': 95,
    'CMD_CAN_DEVICE_SCAN': 96,
    'CMD_CAN_DRV_HARD_PARAMS': 97,
    'CMD_CAN_DRV_STATE': 98,
    'CMD_CAN_DRV_CALIBRATE': 99,
    'CMD_READ_RC_INPUTS': 100,
    'CMD_REALTIME_DATA_CAN_DRV': 101,
    'CMD_EVENT': 102,
    'CMD_READ_PARAMS_EXT3': 104,
    'CMD_WRITE_PARAMS_EXT3': 105,
    'CMD_EXT_IMU_DEBUG_INFO': 106,
    'CMD_SET_DEBUG_PORT': 249,
    'CMD_MAVLINK_INFO': 250,
    'CMD_MAVLINK_DEBUG': 251,
    'CMD_DEBUG_VARS_INFO_3': 253,
    'CMD_DEBUG_VARS_3': 254,
    'CMD_ERROR': 255
}


In [15]:
def as_int(b):
    """ converts b (a bytearray) into a signed integer
        assumes little endian """
    return int.from_bytes(b, byteorder='little', signed=True)

def as_uint(b):
    """ converts b (a bytearray) into an unsigned integer
        assumes little endian """
    return int.from_bytes(b, byteorder='little', signed=False)

In [24]:
def send_data(command_ID, data=None, data_size=0):
    """ Compose and send a command to the SimpleBGC
    Inputs:
        command_ID: a string, such as 'CMD_GET_ANGLES'
        data: a list of numpy numbers
        data_size: the number of bytes in data

    Format:
        head:
            start_char = ord('>') = 0x3E. 1u
            command_ID. 1u
            data_size. 1u. Can be 0
            header_checksum =  (command ID + data_size) % 255. 1u

        body:
            array_of_bytes_of_length_data_size
            body_checksum. 1u
    """
    # compose head:
    start_char = np.uint8(ord('>'))
    header_checksum = np.uint8((L[command_ID] + data_size) % 0xFF)
    
    message = bytearray()
    message.append(start_char)
    message.append(L[command_ID])
    message.append(data_size)
    message.append(header_checksum)

    if data_size > 0:
        for d in data:
            if d.nbytes == 1:
                message.append(d)
            elif d.nbytes == 2:
                d_bytes = d.tobytes()
                print('%s\t-> %s'% (d, d_bytes))
                message.append(d_bytes[0])  # working with little endian
                message.append(d_bytes[1])

    ser.write(message)
    
    return message

In [25]:
def send_angle_command(roll, pitch, yaw):
    """ send an angle command to the gimbal
        roll, pitch and yaw are in degrees """

    scaling = 0.02197265625  # units/degree
    message = [
        np.uint8(1),                # CONTROL_MODE = MODE_ANGLE. do this three times for three axis??
        np.int16(0),                # roll speed
        np.int16(roll/scaling),     # roll angle
        np.int16(0),                # pitch speed
        np.int16(pitch/scaling),    # pitch angle
        np.int16(0),                # yaw speed
        np.int16(yaw/scaling)       # yaw angle
    ]
    return send_data('CMD_CONTROL', message, 13)

In [26]:
msg = send_angle_command(-0.02197265625*2, -180, 360)

msg.hex()

0	-> b'\x00\x00'
-2	-> b'\xfe\xff'
0	-> b'\x00\x00'
-8192	-> b'\x00\xe0'
0	-> b'\x00\x00'
16384	-> b'\x00@'


'3e430d50010000feff000000e000000040'

In [27]:
def get_motor_angles():
    """ get the gimbal angle, as measured by the IMU
        units are in degrees """
    msg = send_data('CMD_GET_ANGLES')
    return msg
    
    # the gimbal returns 18 bytes of data
    # 3 axis, each having an angle, target angle and target speed
    gimbal_state = ser.read(18)  # read up to 18 bytes

    # only interested in the angles
    # they each arrive in 14-bit resolution and require scaling
    scaling = 0.02197265625  # scales to degrees
    IMU = {
        'roll': as_int(gimbal_state[0:2]) * scaling,
        'pitch': as_int(gimbal_state[6:8]) * scaling,
        'yaw': as_int(gimbal_state[12:14] * scaling)
    }
    
    return IMU

In [70]:
msg = get_motor_angles()
msg.hex()

'3e490049'

In [21]:
def turn_off_motors():
    return send_data('CMD_MOTORS_OFF')

In [22]:
msg = turn_off_motors()
msg.hex()

'3e6d006d'

In [109]:
x = bytearray()
x.append(np.uint8(0xAA))
x.append(np.uint8(0))
x.append(np.uint8(255))
x.append(np.uint8(0))
x.append(np.uint8(255))
x.append(np.uint8(2))
x.append(np.uint8(4))
x.append(np.uint8(8))
x.append(np.uint8(16))
x.append(np.uint8(32))

ser.write(x)

10

In [110]:
ser.read(3)

b''